# Maps in reflected light

This notebook discusses how to model phase curves and occultation light curves in reflected light.

## Instantiating a reflected light map

Let's begin by instantiating a map in reflected light. We do this by specifying `reflected=True` when calling `starry.Map()`.

In [ ]:
%matplotlib inline

In [ ]:
%run notebook_setup.py

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import colors
import numpy as np
import starry

starry.config.lazy = False
starry.config.quiet = True

In [ ]:
map = starry.Map(ydeg=15, reflected=True)

Before we set any spherical harmonic coefficients, let's take a look at our map. We can call the ``show()`` method as usual:

In [ ]:
map.show()

By default, the illumination source is along the $+\hat{z}$ direction at $z = 1$, so directly in front of the object, one unit away. In this case, the `starry` normalization results in a sub-illumination intensity of $1/\pi$, a value that falls off as the cosine of the viewing angle to zero at the limb. (As we'll see below, this results in the integral of the illumination over the disk to be equal to $2/3$, the geometric albedo of a Lambert sphere.)

Looking at the figure above, you can easily tell that points in the center of the map (where it is noon) are brighter than points along the edges (where it is dawn or dusk). To change the location of the illumination source, we edit the `xs`, `ys`, and `zs` keywords, just as we do when calling the `flux()` method. These are the Cartesian coordinates of the illumination source.

In [ ]:
map.show(xs=1, ys=0, zs=0)

We are now viewing a uniform map illuminated from the side. The intensity on the left half is zero, since it is completely unilluminated.

By the way, we can see the *albedo* map of the body by specifying `illuminate=False` in the call to `show`:

In [ ]:
norm = colors.Normalize(vmin=0, vmax=1.25)
map.show(illuminate=False, colorbar=True, norm=norm)

By default, the albedo is unity everywhere.

## Normalization and units

The distance between the body and the source in units of the body's radius, $r_s = \sqrt{x_s^2 + y_s^2 + z_s^2}$, controls the overall amplitude of the intensity on the surface and the total flux from the body. We can check that it follows the expected one-over-r-squared law:

In [ ]:
r = np.logspace(0, 2)
plt.figure(figsize=(12, 5))
plt.plot(r, map.intensity(lat=0, lon=0, xs=0, ys=0, zs=r).reshape(-1), label="measured")
plt.plot(r, 1 / (np.pi * r ** 2), label=r"$I(r) = \frac{1}{\pi r_s^2}$", ls="--")
plt.plot(1, 1 / np.pi, "ko")
plt.axvline(1, color="k", ls="--", lw=1, alpha=0.5)
plt.axhline(1 / np.pi, color="k", ls="--", lw=1, alpha=0.5)
plt.yscale("log")
plt.xscale("log")
plt.legend(fontsize=18)
plt.xlabel("star-planet distance", fontsize=24)
plt.ylabel("substellar intensity", fontsize=24);

In particular, the reflected **intensity** (`map.intensity`) of a uniform body at a point on the surface an angle $\phi$ away from the sub-stellar point is given by

$$
I(\phi) = \frac{A}{\pi r_s^2} \, \mathrm{max}(0, \cos\phi)
$$

where $A$ is the body's (spherical) albedo, which we expand in terms of spherical harmonics. This quantity is *unitless*, and that factor of $\pi$ ensures the proper normalization for a Lambert sphere (see below).

Now, the **flux** (`map.flux`) measured from the source is the surface integral of the intensity over the visible portion of the sky-projected disk, so it also scales in the same way. For reference, let's compute the phase curve of a uniform, unit-albedo body at unit distance from the illumination source, seen edge-on over a full year. Here's a movie of what we're computing:

In [ ]:
theta = np.linspace(0, 2 * np.pi, 50)
map.show(xs=np.sin(theta), ys=0, zs=np.cos(theta))

And here's the corresponding light curve:

In [ ]:
plt.figure(figsize=(12, 5))
phase = np.linspace(0, 1, 1000)
theta = np.linspace(0, 2 * np.pi, 1000)
plt.plot(phase, map.flux(xs=np.sin(theta), ys=0, zs=np.cos(theta)))
plt.axhline(0, color="C3", ls="--")
plt.axhline(2 / 3, color="C3", ls="--")
plt.xlabel("orbital phase")
plt.ylabel("reflected planet flux");

Note, in particular, the minimum and maximum values (dashed red lines). When the planet is in a transiting configuration (phase = 0.5), the total flux is zero, since only the nightside is visible. When the planet is at secondary eclipse, the intensity convention described above means that the total flux returned by `starry` is $2/3$.
This value is precisely the geometric albedo of a Lambertian reflector whose spherical albedo is unity (see, e.g., Section 4.5 of [Schwartz & Cowan 2015](https://ui.adsabs.harvard.edu/abs/2015MNRAS.449.4192S/abstract)).

## Setting the albedo

Moving on, reflected light maps behave in a similar way as regular spherical harmonic maps, except the spherical harmonic coefficients `y` represent the expansion of the surface *albedo* rather than *emissivity*. Let's load the continental map of the Earth and look at the albedo distribution:

In [ ]:
map.load("earth", sigma=0.075)

In [ ]:
map.show(projection="moll", illuminate=False, res=500, colorbar=True)

The image we loaded is a grayscale image with unit dynamic range: the oceans have zero albedo and land has unit albedo. This isn't true of the real Earth, whose continents have an albedo closer to 0.4 on average (although the exact value depends on wavelength). 

To fix this, we can scale the map amplitude:

In [ ]:
map.amp *= 0.4

Here's the new albedo distribution, which is more realistic (although we're still assuming a cloudless planet):

In [ ]:
map.show(projection="moll", illuminate=False, res=500, colorbar=True)

Now let's give the map the same obliquity as the Earth...

In [ ]:
map.obl = 23.5

... and view the half-Earth rotating over one cycle:

In [ ]:
map.show(theta=np.linspace(0, 360, 50), xs=1, ys=0, zs=0)

The above animation corresponds to the (northern) winter solstice. Here's the phase curve of the Earth over one rotation at 8 different illumination phases:

In [ ]:
fig = plt.figure(figsize=(12, 8))
theta = np.linspace(0, 360, 1000)
phis = np.linspace(0, 360, 9)[:-1]
xs = np.cos((phis - 90) * np.pi / 180)
zs = -np.sin((phis - 90) * np.pi / 180)
for n, phi in enumerate(phis):
    plt.plot(theta, map.flux(theta=theta, xs=xs[n], ys=0, zs=zs[n]), label=phi)
plt.xlim(0, 360)
plt.ylim(-0.01, 0.15)
plt.xlabel(r"$\theta$ [degrees]", fontsize=24)
plt.ylabel("Flux", fontsize=24)
legend = plt.legend(
    loc="center left", bbox_to_anchor=(1, 0.5), fontsize=36, frameon=False
)
for text in legend.get_texts():
    text.set_color("w")
cmap = plt.get_cmap("plasma")
cmap.set_under("#000000")
for n in range(8):
    ax = fig.add_axes((1.05, 0.775 - 0.087 * n, 0.05, 0.075))
    img = map.render(res=100, xs=xs[n], ys=0, zs=zs[n])
    ax.imshow(img, cmap=cmap, origin="lower", vmin=1e-5, vmax=0.4 / np.pi)
    ax.axis("off")
plt.suptitle("Light curves at different illumination phases", fontsize=24);

## Modeling exoplanet systems

Here's the phase curve of the Earth over one year in orbit around the Sun:

In [ ]:
sun = starry.Primary(starry.Map())
earth = starry.Secondary(map, porb=365.0, prot=1.0, m=0.0, inc=60)
earth.map.inc = earth.inc = 60
sys = starry.System(sun, earth)

t = np.linspace(0, 365.0, 1000)
plt.figure(figsize=(12, 5))
plt.plot(t, sys.flux(t))
plt.xlabel("time [days]")
plt.ylabel("reflected planet flux");